In [1]:
import pytz

import numpy    as np
import pandas   as pd

from dataloader  import upload_trading_idea


# Convert to timframe format from string to int in minute
def process_timeframe(row):
    try:
        if isinstance(row, str):
            for k, s in zip(["H", "D", "W", "M", "S"], [1, 24, 5*24, 5*24*30, 1.]):
                if k in row:
                    row = row.split(k)[0]
                    if isinstance(row, str) and row == '':
                        row = 1
                    row = int(row)*s*60*60
                    break
        return int(row)
    except:
        return 0


path_to_data    = "/home/virgile/Desktop/Trading/Forex Trading/SH_base/tradingview_data_ADAUSDT.csv"
market          = "crypto"

data    = pd.read_csv(path_to_data)   # data pandas.DataFrame

# Rename the columns to fit the database columns name
data    = data.rename(columns={"Pdate": "date", "Acc_Type": "account_type", "Symbol": "code", "Language": "region"})
data    = data.rename(columns={k: k.lower() for k in data.columns})                         # Set all the columns name to lower case

data    = data[["title", "author", "signal", "tag",
                "account_type", "label", "code", "region",
                "description", "url", "date", "timeframe",
                "likes", "comments"]]

# Convert to date string to pandas datetime
data["market"]      = market                                                                # Add market type columns e.g. crypt, stock, forex, etc...
data["date"]        = pd.to_datetime(data["date"], errors='coerce')
data_kept           = data.loc[data["date"].dropna().index]                                 # Get rid of nan due to bad convertion of date to datetime

data_kept['date']       = data_kept['date'].apply(lambda x: pytz.timezone('America/Chicago').localize(x))   # Set datetime to Chicago (US) time zone
data_kept["timestamp"]  = data_kept.apply(lambda x: int(x["date"].timestamp()), axis=1)                     # Gather unix timestamp for all date
data_kept["date"]       = data_kept["date"].dt.strftime('%Y-%m-%d %H:%M:%S%z')
data_kept["timeframe"]  = data_kept.apply(lambda x: process_timeframe(x["timeframe"]), axis=1)              # Convert timeframe columns to second

data_kept.replace([np.inf, -np.inf, np.nan], np.nan, inplace=True)
data_kept_2 = data_kept.dropna()

# Upload the pandas dataframe to the database
failed  = upload_trading_idea(data_kept_2)
print(failed)

print(len(data), len(data_kept), len(data_kept_2))


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 5 seconds...
1000 500
ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 5 seconds...
500 250
200 OK
ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 5 seconds...
250 125
200 OK
200 OK
ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 5 seconds...
500 250
ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 5 seconds...
250 125
200 OK
200 OK
ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 5 seconds...
250 125
200 OK
200 OK
ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection wi

ValueError: No objects to concatenate